In [20]:
import pandas as pd
import numpy as np
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from assets.confint import classification_confint
from sklearn.tree import tree
from assets.treeviz import tree_print
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from joblib import dump

In [21]:
print(sklearn.__version__)

0.21.3


In [22]:
from sklearn.datasets import load_digits

In [4]:
digits = load_digits()
print(digits.data.shape)

(1797, 64)


In [5]:
print(digits.data[:5])

[[0.00 0.00 5.00 13.00 9.00 1.00 0.00 0.00 0.00 0.00 13.00 15.00 10.00
  15.00 5.00 0.00 0.00 3.00 15.00 2.00 0.00 11.00 8.00 0.00 0.00 4.00
  12.00 0.00 0.00 8.00 8.00 0.00 0.00 5.00 8.00 0.00 0.00 9.00 8.00 0.00
  0.00 4.00 11.00 0.00 1.00 12.00 7.00 0.00 0.00 2.00 14.00 5.00 10.00
  12.00 0.00 0.00 0.00 0.00 6.00 13.00 10.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 12.00 13.00 5.00 0.00 0.00 0.00 0.00 0.00 11.00 16.00
  9.00 0.00 0.00 0.00 0.00 3.00 15.00 16.00 6.00 0.00 0.00 0.00 7.00
  15.00 16.00 16.00 2.00 0.00 0.00 0.00 0.00 1.00 16.00 16.00 3.00 0.00
  0.00 0.00 0.00 1.00 16.00 16.00 6.00 0.00 0.00 0.00 0.00 1.00 16.00
  16.00 6.00 0.00 0.00 0.00 0.00 0.00 11.00 16.00 10.00 0.00 0.00]
 [0.00 0.00 0.00 4.00 15.00 12.00 0.00 0.00 0.00 0.00 3.00 16.00 15.00
  14.00 0.00 0.00 0.00 0.00 8.00 13.00 8.00 16.00 0.00 0.00 0.00 0.00
  1.00 6.00 15.00 11.00 0.00 0.00 0.00 1.00 8.00 13.00 15.00 1.00 0.00
  0.00 0.00 9.00 16.00 16.00 5.00 0.00 0.00 0.00 0.00 3.00 13.00 16.00
  16.00 11.00 5.00 0.0

In [6]:
print(digits.target[:5])

[0 1 2 3 4]


In [7]:
#examine shape of images(metrix)
print(digits.images.shape)
#examine shape target
print(digits.target.shape)
images = digits.data
labels = digits.target

(1797, 8, 8)
(1797,)


In [8]:
#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33, random_state=42)

In [9]:
#dtree = tree.DecisionTreeClassifier(criterion='entropy')

In [10]:
#dtree.fit(images,labels)

In [11]:
images = images.reshape((images.shape[0], -1))
images.shape

(1797, 64)

In [12]:
# set up the model with k=3
model = KNeighborsClassifier(n_neighbors=3)

# do train and test
train_X, test_X, train_y, test_y = train_test_split(images, labels, train_size=0.8, test_size=0.2)
model.fit(train_X, train_y)
predict_y = model.predict(test_X)
acc = accuracy_score(test_y, predict_y)
lb, ub = classification_confint(acc, test_X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy: 0.99 (0.98, 1.00)


In [13]:
#with decision tree
dtree = tree.DecisionTreeClassifier(criterion='entropy')
dtree.fit(images,labels)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [14]:
# decision trees
model2 = DecisionTreeClassifier(random_state=1)

In [15]:
# grid search
param_grid = {'max_depth': list(range(1,21)), 'criterion': ['entropy','gini'] }
grid = GridSearchCV(model2, param_grid, cv=5)
grid.fit(images, labels)
print("Grid Search: best parameters: {}".format(grid.best_params_))

Grid Search: best parameters: {'criterion': 'entropy', 'max_depth': 12}


C:\Users\Korak\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(images)
acc = accuracy_score(labels, predict_y)
lb,ub = classification_confint(acc,images.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Accuracy: 1.00 (1.00,1.00)


In [17]:
dump(best_model, 'DTreeModel.joblib')

['DTreeModel.joblib']

In [18]:
# KNN
model = KNeighborsClassifier()

# grid search
param_grid = {'n_neighbors': list(range(1,51))}
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(images, labels)
print("Grid Search: best parameters: {}".format(grid.best_params_))

Grid Search: best parameters: {'n_neighbors': 2}


In [19]:
# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(images)
acc = accuracy_score(labels, predict_y)
lb,ub = classification_confint(acc,images.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Accuracy: 0.99 (0.99,1.00)
